# 数字信号处理

## 使用说明

1. 使用[Julia语言](https://cn.julialang.org/)编写代码，[Jupyter笔记本](https://jupyter.org/)作为交互界面；使用[nbviewer](https://nbviewer.jupyter.org/)进行[快速预览](https://nbviewer.jupyter.org/github/miRoox/HIT-DigitalSignalProcessing-Postgraduate/blob/master/work.ipynb?flush_cache=true)，[Binder](https://mybinder.org/)提供[在线交互](https://mybinder.org/v2/gh/miRoox/HIT-DigitalSignalProcessing-Postgraduate/master?filepath=work.ipynb)。
2. 在Jupyter笔记本中运行代码：选中要运行的代码块，按组合键 <kbd>Shift</kbd>+<kbd>Enter</kbd> 即可运行。也可以通过菜单栏的 "Cell" 下的 "Run Cells" 等菜单项来运行。
3. 首次运行代码时，Julia需要进行即时编译预热，响应速度会比较慢，再次运行速度就正常了。
4. [Binder](https://mybinder.org/)提供的在线交互是通过连接云服务器上运行的内核实现的，在长期不运行的情况下会连接超时，可以点击右边的[链接](https://mybinder.org/v2/gh/miRoox/HIT-DigitalSignalProcessing-Postgraduate/master?filepath=work.ipynb)重新进入。
5. 对于不是自己的笔记本，Jupyter总是把它们当作“不可信”的，这会导致输出的图像等不显示。可以通过菜单栏的 "File" 下的 "信任笔记" 以信任状态重新加载，这时输出的图像等内容就会完整的显示了。

载入包

* [Plots.jl](http://docs.juliaplots.org/latest/) - 绘图
* [Images.jl](https://juliaimages.org/latest/) - 图像处理
* [TestImages.jl](https://testimages.juliaimages.org/) - 测试图像
* [FFTW.jl](https://juliamath.github.io/FFTW.jl/latest/) - 快速傅里叶变换
* [Deconvolution.jl](https://juliadsp.github.io/Deconvolution.jl/dev/) - 反卷积

In [1]:
using Plots, Images, TestImages, FFTW, Deconvolution

运动模糊模型
$$
g(x,y)=f(x,y)*h(x,y)+n(x,y)
$$

其中，
$$
h(x,y)=\begin{cases}
        \frac{1}{L} & 0\le x\le L\cos\theta, 0\le y\le x\tan\theta\\
        0 & \text{其它}
    \end{cases}
$$

In [2]:
function motionPSF(l::Real,θ::Real)
    ker=centered(zeros((2*ceil.(Int,l.*abs.([sin(θ),cos(θ)])).+1)...))
    for i in 0:l
        ker[round.(Int,i.*[-sin(θ),cos(θ)])...]=1/l
    end
    ker
end

motionPSF (generic function with 1 method)

In [3]:
motionPSF(10,7π/18)

OffsetArray(::Array{Float64,2}, -10:10, -4:4) with eltype Float64 with indices -10:10×-4:4:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.1  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.1  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.1  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.1  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.1  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.

In [ ]:
origin=testimage("lena")